In [8]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
#4/AAB7t09pV-WSRzHDG9WFfneYyEWpjiULxtVhTc7s5GH1p216RGD7Dgw
#4/AAA9HQIkK7T5WM10fPa9bzri0trmCXaVmXD8NlLqs8ZWLG93Up4ZXzw

In [0]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.models import load_model
from keras.utils.np_utils import to_categorical
import re

In [0]:
data = pd.read_csv('drive/Colab Notebooks/data.txt')
data.columns = ['cumle','sinif']
data = data[['cumle','sinif']]

In [0]:
# Veri setimizin işlenebilmesi için text verileri numaralara çevirmemiz gerekir.
# Keras bu işlem için hazır bir mekanizma sunmaktadır.
# Tokenizer sınıfı data içerisinde verilen cümleleri analiz ederek. Kelimelerin sıklıklarını hesaplar.
# Parameter: num_words = En sık geçen 25000 kelimeye odaklan. Diğerleri önemli değil
tokenizer = Tokenizer(split=' ',num_words=25000)
# Her bir kelimenin sıklığını(frekansını) hesaplar.
tokenizer.fit_on_texts(data['cumle'].values)
# Tüm cümleler tam sayı dizisine dönüştürülür.
X = tokenizer.texts_to_sequences(data['cumle'].values)
# Bütün metinlerimiz 400 sütundan oluşan bir dizi ile temsil edilecek.
# Çok kısa metinler 0'lar ile doldurulacak. Çok uzun metinler ise kesilecek.
X = pad_sequences(X,maxlen=400)

In [16]:
embed_dim = 128
lstm_out = 128
def build_model():
    model = Sequential()
    # Her bir kelimenin temsil edileceği vektör boyutu. Bu örnek için her bir kelime 128 boyutunda
    # bir vektör ile temsil edilir.
    model.add(Embedding(25000, embed_dim,input_length = X.shape[1], dropout=0.2))
    model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
    model.add(Dense(4,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

  # Çıktılarımızı kategorik hale getirdik. (Opsiyonel)
Y = pd.get_dummies(data['sinif']).values
Y[0]

array([1, 0, 0, 0], dtype=uint8)

In [22]:
# Verinin %90'i train, %10'si test verisi olacak şekilde ayrılır.
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.1, random_state = 1)

model = build_model()
# Oluşturulan model train verileri ile eğitilir. Yapay Sinir Ağı eğitilmeye başlar.
# nb_epoch: İterasyon sayısı
model.fit(X_train, Y_train, nb_epoch = 5, batch_size=32, verbose = 2)
# Train verileri ile model eğitildikten sonra test dataları ile doğruluk oranlarına bakılır.
score = model.evaluate(X_test, Y_test, verbose = 2)
print("score: %.2f" % (score[1]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.2)`
  
/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/5
 - 104s - loss: 1.1521 - acc: 0.5460
Epoch 2/5
 - 103s - loss: 0.3329 - acc: 0.9104
Epoch 3/5
 - 102s - loss: 0.1021 - acc: 0.9732
Epoch 4/5
 - 102s - loss: 0.0458 - acc: 0.9906
Epoch 5/5
 - 102s - loss: 0.0281 - acc: 0.9927
score: 0.93


In [0]:
# 0: Kızmak, 1: Korkmak, 2: Mutluluk, 3:Üzüntü

my_text = ["harca kardeş üzül şu erken doğ çok erken"]

# Verilen örnekler Tokenizer yapısı ile tam sayı dizisine dönüştürülür
# Daha sonra eğitilen modele sırayla verilerek anlam analizi sonuçları elde edilir.
# Her Cümlenin yüzde kaç olumlu ve olumsuz olduğuna dair bilgiler çıktı olarak verilir.
sequences = tokenizer.texts_to_sequences(my_text)
data = pad_sequences(sequences, maxlen=400)
predictions = model.predict(data)
print(predictions)

[[6.4506195e-03 5.3389353e-04 1.8809482e-02 9.7420597e-01]]
